#1. Load CSV data

In [1]:
%tensorflow_version 2.x
import functools
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.2.0-rc2


In [0]:
TRAIN_DATA_URL = 'https://storage.googleapis.com/tf-datasets/titanic/train.csv'
TEST_DATA_URL = 'https://storage.googleapis.com/tf-datasets/titanic/eval.csv'

train_file_path = tf.keras.utils.get_file('train.csv', TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file('eval.csv', TEST_DATA_URL)

In [0]:
# Make numpy values easier to read
np.set_printoptions(precision=3, suppress=True)

##1.1. Load data

In [4]:
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


In [0]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

# read the CSV data from the file and create a dataset
# num_epochs: An int specifying the number of times this dataset is repeated
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path, 
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value='?',
      num_epochs=1,
      ignore_errors=True,
      **kwargs
  )
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [8]:
# Each item in the dataset is a batch
# And a batch size is predefined with 5
# take(number): representing the number of elements of this dataset that should be taken to form the new dataset. --> 1 batch 5 Elements
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print('{:20s}: {}'.format(key, value.numpy()))

show_batch(raw_train_data)
# If the file you are working with does not contain the column names in the first line, pass them in a list of strings to the column_names
# use select_columns to omit the unnessary columns

sex                 : [b'male' b'male' b'male' b'male' b'female']
age                 : [20. 28. 28. 26. 38.]
n_siblings_spouses  : [1 0 0 0 0]
parch               : [0 0 0 0 0]
fare                : [ 7.925  8.05  10.5   56.496 80.   ]
class               : [b'Third' b'Third' b'Second' b'Third' b'First']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'B']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton' b'unknown']
alone               : [b'n' b'y' b'y' b'y' b'y']


In [9]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']
temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)
show_batch(temp_dataset)

sex                 : [b'female' b'female' b'male' b'male' b'male']
age                 : [25.  31.  31.  56.  40.5]
n_siblings_spouses  : [1 0 0 0 0]
parch               : [1 0 0 0 2]
fare                : [30.     7.854 13.    26.55  14.5  ]
class               : [b'Second' b'Third' b'Second' b'First' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'n' b'y' b'y' b'y' b'n']


In [10]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']
temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)
show_batch(temp_dataset)

age                 : [36. 28. 32. 11. 24.]
n_siblings_spouses  : [1 0 0 1 0]
class               : [b'First' b'First' b'Third' b'First' b'Third']
deck                : [b'C' b'unknown' b'E' b'B' b'G']
alone               : [b'n' b'y' b'y' b'n' b'n']


##1.2. Data Preprocessing

###1.2.1. Continuous data

In [11]:
# if your data is already in an appropriate numeric format, you can pack the data into a vector before passing it off to the model.
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [28. 59. 28. 58. 22.]
n_siblings_spouses  : [1. 0. 1. 0. 0.]
parch               : [0. 0. 2. 0. 0.]
fare                : [133.65    7.25   23.45  146.521   7.896]


In [12]:
example_batch, labels_batch = next(iter(temp_dataset)) 
print(example_batch, labels_batch)

OrderedDict([('age', <tf.Tensor: shape=(5,), dtype=float32, numpy=array([ 9. , 14.5,  2. , 29. , 32. ], dtype=float32)>), ('n_siblings_spouses', <tf.Tensor: shape=(5,), dtype=float32, numpy=array([5., 1., 0., 1., 0.], dtype=float32)>), ('parch', <tf.Tensor: shape=(5,), dtype=float32, numpy=array([2., 0., 1., 0., 0.], dtype=float32)>), ('fare', <tf.Tensor: shape=(5,), dtype=float32, numpy=array([46.9  , 14.454, 10.462, 66.6  , 30.5  ], dtype=float32)>)]) tf.Tensor([0 0 0 0 1], shape=(5,), dtype=int32)


In [13]:
# tf.stack stack the dataset according to the defined axis. 
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[ 24.      0.      0.      7.142]
 [ 18.      0.      0.      8.3  ]
 [ 38.      0.      1.    153.462]
 [ 28.      0.      0.     30.696]
 [ 28.      0.      2.      7.75 ]]

[1 0 0 0 0]


In [14]:
# tf.feature_column can handle mixed datatypes, but this incurs some overhead and 
# should be avoided unless really necessary
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'male' b'male' b'male']
age                 : [34. 64. 17. 27. 42.]
n_siblings_spouses  : [0 1 0 0 1]
parch               : [0 4 0 0 0]
fare                : [ 26.55  263.      8.663   7.896  27.   ]
class               : [b'First' b'First' b'Third' b'Third' b'Second']
deck                : [b'unknown' b'C' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'n' b'y' b'y' b'n']


In [0]:
example_batch, labels_batch = next(iter(temp_dataset))

In [0]:
# So define a more general preprocessor that selects a list of numeric features
# and packs them into a single column
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    # casts x (Tensor), x.values (SparseTensor, IndexedSlices) to dtype
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

# __init__(): is used to initialise newly created object
# class Foo:
#   def __init__(self, a, b, c): ...
# x = Foo(1, 2, 3)

# __call__(): implements function call operator
# class Foo:
#   def __call__(self, a, b, c): ...
# x = Foo()
# x(1, 2, 3)

In [0]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

# tf.data.Dataset.map(map_func, num_parallel_calls=None)
# This transformation applies map_func to each element of this dataset, and
# returns a new dataset containing the transformed elements.
# num_parallel_calls: A tf.int32 scalar tf.Tensor, representing the number elements
# to process asynchronously in parallel. If not specified, elements will be processed sequentially.  

In [18]:
show_batch(packed_train_data)

sex                 : [b'male' b'male' b'male' b'female' b'male']
class               : [b'Second' b'Third' b'First' b'Second' b'Second']
deck                : [b'unknown' b'unknown' b'C' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'y' b'n' b'y' b'y']
numeric             : [[ 16.     0.     0.    10.5 ]
 [ 21.     0.     0.     8.05]
 [ 19.     3.     2.   263.  ]
 [ 35.     0.     0.    21.  ]
 [ 35.     0.     0.    10.5 ]]


In [0]:
example_batch, labels_batch = next(iter(packed_train_data))

###1.2.2. Data Normalization for numerical data


In [20]:
# Continuous data should always be normalized.
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [0]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [0]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [23]:
# Partial functions allow us to fix a certain number of arguments of 
# a function and generate a new function. 

# functools.partial calls normalize_numeric function, with mean=MEAN, and std=STD
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

# Represents real valued or numerical features Returns: A NumericColumn
numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7f796a8f82f0>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [24]:
# When you train the model, include this feature column to select and center this block of numeric data
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[28.   ,  1.   ,  0.   , 82.171],
       [39.   ,  0.   ,  0.   , 26.   ],
       [39.   ,  0.   ,  0.   , 24.15 ],
       [ 4.   ,  3.   ,  2.   , 27.9  ],
       [28.   ,  0.   ,  0.   ,  7.55 ]], dtype=float32)>

In [25]:
# A layer that produces a dense Tensor based on given feature_columns
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.13 ,  0.395, -0.479,  0.875],
       [ 0.749, -0.474, -0.479, -0.154],
       [ 0.749, -0.474, -0.479, -0.187],
       [-2.049,  2.132,  2.043, -0.119],
       [-0.13 , -0.474, -0.479, -0.492]], dtype=float32)

###1.2.3. One-hot exchange for categorical data

In [0]:
# one-hot for categorical data
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [27]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [28]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.]


###1.2.4. Combined preprocessing data (Layer)

In [29]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)
print(preprocessing_layer(example_batch).numpy()[0])

[ 0.     1.     1.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     1.     0.     0.    -0.13   0.395
 -0.479  0.875  0.     1.   ]


##1.3. Build the model

In [0]:
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy']
)

In [31]:
# Train, evaluate, and predict
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 0s 2ms/step - loss: 0.5024 - accuracy: 0.7544
Epoch 2/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4194 - accuracy: 0.8278
Epoch 3/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4034 - accuracy: 0.8166
Epoch 4/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3897 - accuracy: 0.8437
Epoch 5/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3771 - accuracy: 0.8293
Epoch 6/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3694 - accuracy: 0.8469
Epoch 7/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3592 - accuracy: 0.8453
Epoch 8/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3557 - accuracy: 0.8533
Epoch 9/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3446 - accuracy: 0.8469
Epoch 10/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3351 - accuracy: 0.8565

In [32]:
test_loss, test_accuracy = model.evaluate(test_data)
print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

53/53 [==============================] - 0s 1ms/step - loss: 0.4718 - accuracy: 0.8447


Test Loss 0.4718002378940582, Test Accuracy 0.8446969985961914


In [35]:
predictions = model.predict(test_data)
print(predictions[:10])
# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

[[-1.372]
 [-0.307]
 [-0.965]
 [ 3.122]
 [-6.683]
 [-1.404]
 [10.205]
 [ 2.908]
 [-0.965]
 [ 0.522]]
Predicted survival: 20.23%  | Actual outcome:  SURVIVED
Predicted survival: 42.39%  | Actual outcome:  DIED
Predicted survival: 27.60%  | Actual outcome:  DIED
Predicted survival: 95.78%  | Actual outcome:  DIED
Predicted survival: 0.13%  | Actual outcome:  DIED


#2. Load NumPy Data

loading data from NumPy arrays into a tf.data,Dataset

In [0]:
%tensorflow_version 2.x
import numpy as np
import tensorflow as tf

In [37]:
DATA_URL = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz'

path = tf.keras.utils.get_file('mnist.npz', DATA_URL)
# np.load: .npy, .npz
with np.load(path) as data:
  train_examples = data['x_train']
  train_labels = data['y_train']
  test_examples = data['x_test']
  test_labels = data['y_test']

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
# Load NumPy arrays with tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_examples, test_labels))
# return a tf.data.Dataset

##2.1. Use the datasets

In [0]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

# A batch contains 64 elements/objects
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

##2.2. Build and Train a model

In [40]:
model = tf.keras.Sequential(
    [
     tf.keras.layers.Flatten(input_shape=(28, 28)), 
     tf.keras.layers.Dense(128, activation='relu'),
     tf.keras.layers.Dense(10)
    ]
)

model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['sparse_categorical_accuracy'])

model.fit(train_dataset, epochs=10)

Epoch 1/10
938/938 [==============================] - 3s 3ms/step - loss: 3.1503 - sparse_categorical_accuracy: 0.8766
Epoch 2/10
938/938 [==============================] - 3s 3ms/step - loss: 0.5315 - sparse_categorical_accuracy: 0.9267
Epoch 3/10
938/938 [==============================] - 3s 3ms/step - loss: 0.3961 - sparse_categorical_accuracy: 0.9445
Epoch 4/10
938/938 [==============================] - 3s 3ms/step - loss: 0.3264 - sparse_categorical_accuracy: 0.9519
Epoch 5/10
938/938 [==============================] - 3s 3ms/step - loss: 0.2921 - sparse_categorical_accuracy: 0.9588
Epoch 6/10
938/938 [==============================] - 3s 3ms/step - loss: 0.2686 - sparse_categorical_accuracy: 0.9620
Epoch 7/10
938/938 [==============================] - 3s 3ms/step - loss: 0.2343 - sparse_categorical_accuracy: 0.9670
Epoch 8/10
938/938 [==============================] - 3s 3ms/step - loss: 0.2198 - sparse_categorical_accuracy: 0.9686
Epoch 9/10
938/938 [============================

In [41]:
model.evaluate(test_dataset)

157/157 [==============================] - 0s 1ms/step - loss: 0.6510 - sparse_categorical_accuracy: 0.9531


[0.6510223150253296, 0.9531000256538391]

#3. Load a pandas.DataFrame

loading pandas dataframes into tf.data.Dataset

In [0]:
# Read data using pandas
%tensorflow_version 2.x
import pandas as pd
import tensorflow as tf

In [0]:
csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/applied-dl/heart.csv')

In [3]:
df = pd.read_csv(csv_file)
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [4]:
# check the type
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal         object
target        int64
dtype: object

In [0]:
# object to numerical value
df['thal'] = pd.Categorical(df['thal'])
df['thal'] = df.thal.cat.codes

In [6]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,2,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,4,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0


##3.1. Load data using tf.data.Dataset

In [7]:
target = df.pop('target')
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))
for feat, targ in dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [ 63.    1.    1.  145.  233.    1.    2.  150.    0.    2.3   3.    0.
   2. ], Target: 0
Features: [ 67.    1.    4.  160.  286.    0.    2.  108.    1.    1.5   2.    3.
   3. ], Target: 1
Features: [ 67.    1.    4.  120.  229.    0.    2.  129.    1.    2.6   2.    2.
   4. ], Target: 0
Features: [ 37.    1.    3.  130.  250.    0.    0.  187.    0.    3.5   3.    0.
   3. ], Target: 0
Features: [ 41.    0.    2.  130.  204.    0.    2.  172.    0.    1.4   1.    0.
   3. ], Target: 0


In [8]:
# pd.Series implements the __array__ protocl
tf.constant(df['thal'])

<tf.Tensor: shape=(303,), dtype=int8, numpy=
array([2, 3, 4, 3, 3, 3, 3, 3, 4, 4, 2, 3, 2, 4, 4, 3, 4, 3, 3, 3, 3, 3,
       3, 4, 4, 3, 3, 3, 3, 4, 3, 4, 3, 4, 3, 3, 4, 2, 4, 3, 4, 3, 4, 4,
       2, 3, 3, 4, 3, 3, 4, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 4,
       4, 2, 3, 3, 4, 3, 4, 3, 3, 4, 4, 3, 3, 4, 4, 3, 3, 3, 3, 4, 4, 4,
       3, 3, 4, 3, 4, 4, 3, 4, 3, 3, 3, 4, 3, 4, 4, 3, 3, 4, 4, 4, 4, 4,
       3, 3, 3, 3, 4, 3, 4, 3, 4, 4, 3, 3, 2, 4, 4, 2, 3, 3, 4, 4, 3, 4,
       3, 3, 4, 2, 4, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4,
       4, 3, 3, 3, 4, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 3, 2,
       4, 4, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 2, 2, 4, 3, 4, 2, 4, 3,
       3, 4, 3, 3, 3, 3, 4, 3, 4, 3, 4, 2, 2, 4, 3, 4, 3, 2, 4, 3, 3, 2,
       4, 4, 4, 4, 3, 0, 3, 3, 3, 3, 1, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 4,
       3, 3, 4, 4, 4, 4, 3, 3, 4, 3, 4, 3, 4, 4, 3, 4, 4, 3, 4, 4, 3, 3,
      

In [0]:
train_dataset = dataset.shuffle(len(df)).batch(1)

##3.2. Create and train a model

In [10]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

model = get_compiled_model()
model.fit(train_dataset, epochs=15)

Epoch 1/15

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

303/303 [==============================] - 0s 1ms/step - loss: 15.4612 - accuracy: 0.6997
Epoch 2/15
303/303 [==============================] - 0s 1ms/step - loss: 0.9660 - accuracy: 0.7426
Epoch 3/15
303/303 [==============================] - 0s 1ms/step - loss: 0.8204 - accuracy: 0.7129
Epoch 4/15
303/303 [==============================] - 0s 1ms/step - loss: 0.7719 - accuracy: 0.7327
Epoch 5/15
303/303 [==============================] - 0s 1ms/step - loss: 0.6767 - accuracy: 0.7261
Epoch 6

##3.3. Alternative to feature columns



In [11]:
inputs = {key: tf.keras.layers.Input(shape=(), name=key) for key in df.keys()}
print(list(inputs.values()))
x = tf.stack(list(inputs.values()), axis=-1)

x = tf.keras.layers.Dense(10, activation='relu')(x)
output = tf.keras.layers.Dense(1)(x)

model_func = tf.keras.Model(inputs=inputs, outputs=output)
model_func.compile(optimizer='adam',
                   loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=['accuracy'])

[<tf.Tensor 'age:0' shape=(None,) dtype=float32>, <tf.Tensor 'sex:0' shape=(None,) dtype=float32>, <tf.Tensor 'cp:0' shape=(None,) dtype=float32>, <tf.Tensor 'trestbps:0' shape=(None,) dtype=float32>, <tf.Tensor 'chol:0' shape=(None,) dtype=float32>, <tf.Tensor 'fbs:0' shape=(None,) dtype=float32>, <tf.Tensor 'restecg:0' shape=(None,) dtype=float32>, <tf.Tensor 'thalach:0' shape=(None,) dtype=float32>, <tf.Tensor 'exang:0' shape=(None,) dtype=float32>, <tf.Tensor 'oldpeak:0' shape=(None,) dtype=float32>, <tf.Tensor 'slope:0' shape=(None,) dtype=float32>, <tf.Tensor 'ca:0' shape=(None,) dtype=float32>, <tf.Tensor 'thal:0' shape=(None,) dtype=float32>]


In [12]:
dict_slices = tf.data.Dataset.from_tensor_slices((df.to_dict('list'), target.values)).batch(16)
for dict_slice in dict_slices.take(1):
  print(dict_slice)

({'age': <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([63, 67, 67, 37, 41, 56, 62, 57, 63, 53, 57, 56, 56, 44, 52, 57],
      dtype=int32)>, 'sex': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1], dtype=int32)>, 'cp': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 4, 4, 3, 2, 2, 4, 4, 4, 4, 4, 2, 3, 2, 3, 3], dtype=int32)>, 'trestbps': <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([145, 160, 120, 130, 130, 120, 140, 120, 130, 140, 140, 140, 130,
       120, 172, 150], dtype=int32)>, 'chol': <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([233, 286, 229, 250, 204, 236, 268, 354, 254, 203, 192, 294, 256,
       263, 199, 168], dtype=int32)>, 'fbs': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0], dtype=int32)>, 'restecg': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0, 0], dtype=int32)>, 'thalach': <tf.Tensor: shape=(16

In [13]:
model_func.fit(dict_slices, epochs=15)

Epoch 1/15
19/19 [==============================] - 0s 2ms/step - loss: 25.0957 - accuracy: 0.7261
Epoch 2/15
19/19 [==============================] - 0s 2ms/step - loss: 21.6677 - accuracy: 0.7261
Epoch 3/15
19/19 [==============================] - 0s 2ms/step - loss: 18.6807 - accuracy: 0.7261
Epoch 4/15
19/19 [==============================] - 0s 1ms/step - loss: 15.8104 - accuracy: 0.7261
Epoch 5/15
19/19 [==============================] - 0s 2ms/step - loss: 12.8323 - accuracy: 0.7261
Epoch 6/15
19/19 [==============================] - 0s 2ms/step - loss: 9.6094 - accuracy: 0.7261
Epoch 7/15
19/19 [==============================] - 0s 2ms/step - loss: 5.8482 - accuracy: 0.7261
Epoch 8/15
19/19 [==============================] - 0s 2ms/step - loss: 1.8366 - accuracy: 0.6997
Epoch 9/15
19/19 [==============================] - 0s 2ms/step - loss: 1.0809 - accuracy: 0.6568
Epoch 10/15
19/19 [==============================] - 0s 2ms/step - loss: 0.8781 - accuracy: 0.7261
Epoch 11/15
19

#4. Load images

how to load an image dataset using tf.data